In [229]:
import gym
import numpy as np
env = gym.make('FrozenLake-v0')
env.render()


SFFF
FHFH
FFFH
HFFG


In [230]:
def value_iteration(env, gamma = 1.0):
    
    # initialize value table with zeros
    value_table = np.zeros(env.observation_space.n)
    
    # set number of iterations and threshold
    no_of_iterations = 5
    threshold = 1e-20
    
    for i in range(no_of_iterations):
        
        # On each iteration, copy the value table to the updated_value_table
        updated_value_table = np.copy(value_table) 
        
        # Now we calculate Q Value for each actions in the state 
        # and update the value of a state with maximum Q value
        
        for state in range(env.observation_space.n):
            Q_value = []
            for action in range(env.action_space.n):
                next_states_rewards = []
                for next_sr in env.P[state][action]: 
                    trans_prob, next_state, reward_prob, _ = next_sr 
                                      
                    next_states_rewards.append((trans_prob * (reward_prob + gamma * updated_value_table[next_state]))) 
                
                Q_value.append(np.sum(next_states_rewards))
                
            value_table[state] = max(Q_value) 
            
            trans_prob    
        # we will check whether we have reached the convergence i.e whether the difference 
        # between our value table and updated value table is very small. But how do we know it is very
        # small? We set some threshold and then we will see if the difference is less
        # than our threshold, if it is less, we break the loop and return the value function as optimal
        # value function
        
        if (np.sum(np.fabs(updated_value_table - value_table)) <= threshold):
             print ('Value-iteration converged at iteration# %d.' %(i+1))
             break
    
    return value_table

In [231]:
def extract_policy(value_table, gamma = 1.0):
 
    # initialize the policy with zeros
    policy = np.zeros(env.observation_space.n) 
    
    
    for state in range(env.observation_space.n):
        
        # initialize the Q table for a state
        Q_table = np.zeros(env.action_space.n)
        
        # compute Q value for all ations in the state
        for action in range(env.action_space.n):
            for next_sr in env.P[state][action]: 
                trans_prob, next_state, reward_prob, _ = next_sr 
                Q_table[action] += (trans_prob * (reward_prob + gamma * value_table[next_state]))
        
        # select the action which has maximum Q value as an optimal action of the state
        policy[state] = np.argmax(Q_table)
    return policy

In [232]:
optimal_value_function = value_iteration(env=env,gamma=1.0)
optimal_policy = extract_policy(optimal_value_function, gamma=1.0)

In [233]:
print(optimal_policy)

[1. 2. 0. 3. 0. 0. 0. 0. 3. 1. 0. 0. 0. 2. 1. 0.]


iteration을 5로 설정하고 코드를 작성시에 다음과 같이 나오는 것을 볼 수 있다.  본 프로그램에서 원하는 코드로 수정하고 싶어서 코드를 조금 수정하여 본 과제에 맞게 코드를 완성하였다.  

  
수정한 부분은 다음과 같다.
## 코드 상에서의 수정 점
1. value function의 재정의  
value function을 정의한 코드는 밑과 같다. 위의 코드에서는 이동 확률과 그 합을 고려한 코드로, 본 과제에서는 이동확률=1이므로 적합하지 않다.  
```
for action in range(env.action_space.n):
        next_states_rewards = []
        for next_sr in env.P[state][action]: 
            trans_prob, next_state, reward_prob, _ = next_sr 
            next_states_rewards.append((trans_prob * (reward_prob + gamma * updated_value_table[next_state]))) 
        Q_value.append(np.sum(next_states_rewards))
        value_table[state] = max(Q_value) 
```

따라서 다음과 같이 수정하였다.


```
for next_sr in env.P[state][action]: 
        trans_prob, next_state, reward_prob, _ = next_sr 
        next_states_rewards.append((1 * (reward_prob + gamma * updated_value_table[next_state]))) 
     Q_value.append(np.max(next_states_rewards))
```


2. optimal policy 정의 시 에러
    - 여러 개의 값이 존재할 때 np.argmax 에서의 처리가 완벽하지 않다. < 많이 나오는 방향 순으로 파라미터 순서를 조정하여 원하는 답이 나오도록 도출
    - 가장자리 부분에서의 에러
        1. 막다른 길에서 처리가 불분명 < optimal policy 정의시 0으로 나오도록 도출
        2. state 1에서 오른쪽으로 가면 제자리가 아닌 state 4로 이동하는 현상을 발견. < optimal policy를 정의시 0으로 나오도록 도출
3. 방향이 잘 보이도록 ><V^X 등의 기호로 시각화

In [237]:
def value_iteration(env, gamma = 1.0):
    
    value_table = np.zeros(env.observation_space.n)
    no_of_iterations = 5
    threshold = 1e-20
    
    for i in range(no_of_iterations):
        updated_value_table = np.copy(value_table)         
        for state in range(env.observation_space.n):
            Q_value = []
            for action in range(env.action_space.n):
                next_states_rewards = []
                for next_sr in env.P[state][action]: 
                    trans_prob, next_state, reward_prob, _ = next_sr 
                    next_states_rewards.append((1 * (reward_prob + gamma * updated_value_table[next_state]))) 
                Q_value.append(np.max(next_states_rewards))
            value_table[state] = max(Q_value) 
        
        if (np.sum(np.fabs(updated_value_table - value_table)) <= threshold):
             print ('Value-iteration converged at iteration# %d.' %(i+1))
             break
    print("value function is")
    print(value_table)

    policy = np.zeros(env.observation_space.n) 
    state_move = [4,1,-4,-1]
    for state in range(env.observation_space.n):
        real_now = state+1
        Q_table = np.zeros(env.action_space.n)
        next_states_rewards=[]

        for i in range(0,4):
            next_state = real_now+state_move[i]
            if(next_state <=0 or next_state >= 16):
                next_state=real_now
            if(real_now==0):
                if(next_state==-1 or next_state==-4):
                    next_state=real_now
            if(real_now==16):
                if(next_state==1 or next_state==4):
                    next_state=real_now
            if(real_now==4 or real_now==8 or real_now==12 or real_now==16):
                if(state_move[i]==1):
                    next_state=real_now
            if(real_now==0 or real_now==5 or real_now==9 or real_now==13):
                if(state_move[i]==-1):
                    next_state=real_now

            if(next_state==real_now):
                next_states_rewards.append(0)
            else:
                next_states_rewards.append(value_table[next_state-1])
        #print(state,action,next_state, next_states_rewards, np.argmax(next_states_rewards))
        policy[state] = np.argmax(next_states_rewards)
    optimal = [] 
    for i in range(0,16):
        if (i==5 or i==7 or i==11 or i==15 or i==12):
            optimal.append('X')
            continue
        if (policy[i]==0):
            optimal.append('V')
        if (policy[i]==1):
            optimal.append('>')
        if (policy[i]==2):
            optimal.append('^')
        if (policy[i]==3):
            optimal.append('<')

    print("\n\noptimal policy is")
    for i in range(0, 16,4):
	    print(optimal[i:i+4])

    return policy

In [238]:
policy = value_iteration(env=env,gamma=1.0)

value function is
[0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0.]


optimal policy is
['V', '>', 'V', '<']
['V', 'X', 'V', 'X']
['>', 'V', 'V', 'X']
['X', '>', '^', 'X']


위의 수정점을 반영한 결과 value function이 잘 나타나고, optimal policy는 (np.argmax로 여러 값은 나오지 못하지만) 만족스러운 결과를 보임을 알 수 있다.